# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,275.89,97,100,1.28,GS,1699884956
1,1,waitangi,-43.9535,-176.5597,285.83,93,25,2.68,NZ,1699884956
2,2,beyneu,45.3167,55.2000,283.75,42,70,9.71,KZ,1699884956
3,3,port mathurin,-19.6833,63.4167,299.17,72,4,1.08,MU,1699884957
4,4,ushuaia,-54.8000,-68.3000,282.96,53,75,7.72,AR,1699884933



### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Configure the map
map_plot_2 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
)

# Display the map plot
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Define your criteria (e.g., max temperature, humidity, cloudiness, wind speed)
# Filter the DataFrame based on the criteria
city_data_df = city_data_df[
    (city_data_df["Max Temp"] <= 300) & (city_data_df["Cloudiness"] == 40) & (city_data_df["Wind Speed"] <= 19)]
# Drop any rows with null values if needed
city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,10,lompoc,34.6391,-120.4579,289.70,85,40,3.58,US,1699884957
20,20,nova sintra,14.8667,-24.7167,296.80,68,40,5.77,CV,1699884974
21,21,jamestown,42.0970,-79.2353,280.21,72,40,4.63,US,1699884788
28,28,isafjordur,66.0755,-23.1240,277.17,65,40,9.26,IS,1699884976
79,79,kill devil hills,36.0307,-75.6760,286.47,68,40,5.66,US,1699885078
119,119,melchor de mencos,17.0686,-89.1522,297.15,94,40,2.06,GT,1699885120
150,150,rodionovo-nesvetaiskoye,47.6000,39.7000,284.96,75,40,3.82,RU,1699885171
160,160,santa ana,33.7456,-117.8678,292.00,47,40,0.00,US,1699885204
238,238,broome,42.2506,-75.8330,274.10,79,40,2.68,US,1699885500
296,296,makaha,21.4694,-158.2175,298.12,73,40,5.66,US,1699885693


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Create a new DataFrame called hotel_df
hotel_df = city_data_df[['City', 'Country', 'Lat','Lng', 'Humidity']].copy()

# Add an empty "Hotel Name" column to the new DataFrame
hotel_df['Hotel Name'] = ''

# Display sample data from the new DataFrame
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
10,lompoc,US,34.6391,-120.4579,85,
20,nova sintra,CV,14.8667,-24.7167,68,
21,jamestown,US,42.0970,-79.2353,72,
28,isafjordur,IS,66.0755,-23.1240,65,
79,kill devil hills,US,36.0307,-75.6760,68,
119,melchor de mencos,GT,17.0686,-89.1522,94,
150,rodionovo-nesvetaiskoye,RU,47.6000,39.7000,75,
160,santa ana,US,33.7456,-117.8678,47,
238,broome,US,42.2506,-75.8330,79,
296,makaha,US,21.4694,-158.2175,73,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000  # Set to 10,000 meters
params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key
}

print("Starting hotel search")

for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL for the hotel search API
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
nova sintra - nearest hotel: Residência Ka Dencho
jamestown - nearest hotel: DoubleTree Jamestown
isafjordur - nearest hotel: Hótel Horn
kill devil hills - nearest hotel: Mariner Days Inn & Suites
melchor de mencos - nearest hotel: Hotel Central
rodionovo-nesvetaiskoye - nearest hotel: Гостиница
santa ana - nearest hotel: Holiday Inn Express
broome - nearest hotel: No hotel found
makaha - nearest hotel: No hotel found
tolanaro - nearest hotel: Hôtel Mahavokey
mthatha - nearest hotel: No hotel found
happy valley-goose bay - nearest hotel: Hotel North Two


,City,Country,Lat,Lng,Humidity,Hotel Name
10,lompoc,US,34.6391,-120.4579,85,Embassy Suites by Hilton Lompoc Central Coast
20,nova sintra,CV,14.8667,-24.7167,68,Residência Ka Dencho
21,jamestown,US,42.0970,-79.2353,72,DoubleTree Jamestown
28,isafjordur,IS,66.0755,-23.1240,65,Hótel Horn
79,kill devil hills,US,36.0307,-75.6760,68,Mariner Days Inn & Suites
119,melchor de mencos,GT,17.0686,-89.1522,94,Hotel Central
150,rodionovo-nesvetaiskoye,RU,47.6000,39.7000,75,Гостиница
160,santa ana,US,33.7456,-117.8678,47,Holiday Inn Express
238,broome,US,42.2506,-75.8330,79,No hotel found
296,makaha,US,21.4694,-158.2175,73,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.¶

In [8]:
# Configure the map
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    hover_cols=['Hotel Name', 'Country']  # Add additional hover columns
)

# Display the map plot
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)